# Normalização de Dados com Normalizer
### Inicializando o PySpark

In [3]:
import findspark
import pyspark
from pyspark.sql import SparkSession

# Faz a Interafce entre o Spark e o Jupyter Notebook
findspark.init()

# Inicializando uma Sessão no Spark
spark = SparkSession.builder.appName("Normalizer").getOrCreate()

In [19]:
from pyspark.ml.feature import VectorAssembler

### Carregando a Base de Dados usado nesta Aula

In [9]:
carros = spark.read.load("Material_do_Curso/Carros.csv", format="csv",
                        sep=";", inferSchema=True, header=True)
carros.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



### Crianda DataFrame usando VectorAssembler

In [12]:
# Instanciando um Objeto VectorAssembler e aplicando a transformação dos dados
vecassembler = VectorAssembler(inputCols=["Consumo", "Cilindros", "Cilindradas"], outputCol="Matriz")
carros_vetor = vecassembler.transform(carros)
carros_vetor.select("Consumo", "Cilindros", "Cilindradas", "Matriz").show(5)

+-------+---------+-----------+-----------------+
|Consumo|Cilindros|Cilindradas|           Matriz|
+-------+---------+-----------+-----------------+
|     21|        6|        160| [21.0,6.0,160.0]|
|     21|        6|        160| [21.0,6.0,160.0]|
|    228|        4|        108|[228.0,4.0,108.0]|
|    214|        6|        258|[214.0,6.0,258.0]|
|    187|        8|        360|[187.0,8.0,360.0]|
+-------+---------+-----------+-----------------+
only showing top 5 rows



### Normalizando os dados usando o método Normalizer

In [17]:
from pyspark.ml.feature import Normalizer

In [13]:
# Instanciando um Objeto Normalizer e aplicando a transformação dos dados
normalizador = Normalizer(inputCol="Matriz", outputCol="Matriznorm", p=1.0)
carrrosnorn = normalizador.transform(carros_vetor)
carrrosnorn.select("Matriz", "Matriznorm").show(truncate=False)

+------------------+--------------------------------------------------------------+
|Matriz            |Matriznorm                                                    |
+------------------+--------------------------------------------------------------+
|[21.0,6.0,160.0]  |[0.11229946524064172,0.03208556149732621,0.8556149732620321]  |
|[21.0,6.0,160.0]  |[0.11229946524064172,0.03208556149732621,0.8556149732620321]  |
|[228.0,4.0,108.0] |[0.6705882352941176,0.011764705882352941,0.3176470588235294]  |
|[214.0,6.0,258.0] |[0.4476987447698745,0.012552301255230125,0.5397489539748954]  |
|[187.0,8.0,360.0] |[0.33693693693693694,0.014414414414414415,0.6486486486486487] |
|[181.0,6.0,225.0] |[0.4393203883495146,0.014563106796116505,0.5461165048543689]  |
|[143.0,8.0,360.0] |[0.27984344422700586,0.015655577299412915,0.7045009784735812] |
|[244.0,4.0,1467.0]|[0.1422740524781341,0.0023323615160349854,0.8553935860058309] |
|[228.0,4.0,1408.0]|[0.13902439024390245,0.0024390243902439024,0.85853658536

---
 
# Padronização de Dados com StandardScaler
 
### Características
Normaliza os atribultos para o desvio padrão ou média zero;

Parâmetros:
> + **withStd**: Transforma os dados para o desvio padrão da unidade, Padrão: True;
> + **withMean**: Antes de realizar a transformação, centraliza os dados pela média. Padrão: False.

In [18]:
from pyspark.ml.feature import StandardScaler

In [15]:
# Instanciando um Objeto StandardScaler e aplicando a transformação dos dados
standartscaler = StandardScaler(inputCol="Matriz", outputCol="AtandardMatriz", withStd=True, withMean=False)
modelo = standartscaler.fit(carros_vetor)
carrrosstandard = modelo.transform(carros_vetor)
carrrosstandard.select("Matriz", "AtandardMatriz").show(truncate=False)

+------------------+-----------------------------------------------------------+
|Matriz            |AtandardMatriz                                             |
+------------------+-----------------------------------------------------------+
|[21.0,6.0,160.0]  |[0.24996122082808128,3.359609874407659,0.20137542427273997]|
|[21.0,6.0,160.0]  |[0.24996122082808128,3.359609874407659,0.20137542427273997]|
|[228.0,4.0,108.0] |[2.713864683276311,2.239739916271773,0.13592841138409947]  |
|[214.0,6.0,258.0] |[2.5472238693909235,3.359609874407659,0.32471787163979315] |
|[187.0,8.0,360.0] |[2.2258451568976763,4.479479832543546,0.4530947046136649]  |
|[181.0,6.0,225.0] |[2.15442766523251,3.359609874407659,0.28318419038354053]   |
|[143.0,8.0,360.0] |[1.7021168846864583,4.479479832543546,0.4530947046136649]  |
|[244.0,4.0,1467.0]|[2.904311327716754,2.239739916271773,1.8463609213006844]   |
|[228.0,4.0,1408.0]|[2.713864683276311,2.239739916271773,1.7721037336001115]   |
|[192.0,6.0,1676.0]|[2.28535

# Padronização de Dados com RobustScaler

Faz a padronização dos dados levando em consideração o Quantil.

Sua aplicação ajuda a padronizar dados com ***outliers*** se comparado com o StandardScaler.

Parâmetros:
> + **lower**: Quantil inferior usado para o cálculo dos intervalos, possui valor padrão: 0,25 
> + **upper**: Quantil superior usado para o cálculo dos intervalos, possui valor padrão: 0,0.75
> + **withScaling**: Dimensiona os dados para o Quantil, valor booleano, possui valor padrão: True
> + **withCentering**: Centraliza os dados com a mediana antes de aplicar a transformação, valor booleano, possui valor padrão: False

In [22]:
from pyspark.ml.feature import RobustScaler

In [21]:
# Instanciando um Objeto RobustScaler e aplicando a transformação dos dados
robustscaler = RobustScaler(inputCol="Matriz", outputCol="RobustMatriz",
                            withScaling=True, withCentering=False,
                            lower=0.25, upper=0.75)
modelo = robustscaler.fit(carros_vetor)
carrrosrobust = modelo.transform(carros_vetor)
carrrosrobust.select("Matriz", "RobustMatriz").show(truncate=False)

+------------------+---------------------------------------------+
|Matriz            |RobustMatriz                                 |
+------------------+---------------------------------------------+
|[21.0,6.0,160.0]  |[0.29166666666666663,1.5,0.16967126193001061]|
|[21.0,6.0,160.0]  |[0.29166666666666663,1.5,0.16967126193001061]|
|[228.0,4.0,108.0] |[3.1666666666666665,1.0,0.11452810180275717] |
|[214.0,6.0,258.0] |[2.972222222222222,1.5,0.27359490986214213]  |
|[187.0,8.0,360.0] |[2.597222222222222,2.0,0.3817603393425239]   |
|[181.0,6.0,225.0] |[2.513888888888889,1.5,0.23860021208907745]  |
|[143.0,8.0,360.0] |[1.986111111111111,2.0,0.3817603393425239]   |
|[244.0,4.0,1467.0]|[3.388888888888889,1.0,1.555673382820785]    |
|[228.0,4.0,1408.0]|[3.1666666666666665,1.0,1.4931071049840934]  |
|[192.0,6.0,1676.0]|[2.6666666666666665,1.5,1.7773064687168612]  |
|[178.0,6.0,1676.0]|[2.472222222222222,1.5,1.7773064687168612]   |
|[164.0,8.0,2758.0]|[2.2777777777777777,2.0,2.924708377518558]